In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Hàm kiểm tra hai hộp có gần nhau hay không
def boxes_in_vicinity(box1, box2, threshold):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    center1_x = x1 + w1 / 2
    center1_y = y1 + h1 / 2
    center2_x = x2 + w2 / 2
    center2_y = y2 + h2 / 2
    distance = np.hypot(center1_x - center2_x, center1_y - center2_y)

    return distance <= threshold

# Template Matching
def match_template(image_file, template_file):

    match_threshold = 0.09
    proximity_threshold = 50

    img = cv2.imread(image_file)
    tmpl = cv2.imread(template_file)

    # Tiền xử lý
    tmpl_gray = cv2.cvtColor(tmpl, cv2.COLOR_BGR2GRAY)
    _, binary_mask = cv2.threshold(tmpl_gray, 240, 255, cv2.THRESH_BINARY_INV)
    mask_3ch = cv2.merge([binary_mask] * 3)

    tmpl_h, tmpl_w = tmpl.shape[:2]

    # Tách các kênh RGB của ảnh và mẫu
    img_r, img_g, img_b = cv2.split(img)
    tmpl_r, tmpl_g, tmpl_b = cv2.split(tmpl)
    mask_r, mask_g, mask_b = cv2.split(mask_3ch)

    # Áp dụng template matching trên từng kênh
    res_r = cv2.matchTemplate(img_r, tmpl_r, cv2.TM_SQDIFF_NORMED, mask=mask_r)
    res_g = cv2.matchTemplate(img_g, tmpl_g, cv2.TM_SQDIFF_NORMED, mask=mask_g)
    res_b = cv2.matchTemplate(img_b, tmpl_b, cv2.TM_SQDIFF_NORMED, mask=mask_b)

    # Tính trung bình kết quả của các kênh
    combined_result = (res_r + res_g + res_b) / 3

    # Tìm các vị trí có giá trị khớp dưới ngưỡng
    match_locs = np.where(combined_result <= match_threshold)

    # Tạo danh sách các hộp chứa đối tượng khớp
    bounding_rects = [(x, y, tmpl_w, tmpl_h) for x, y in zip(*match_locs[::-1])]

    # Nhóm các hộp gần nhau
    grouped_rects = []
    for rect in bounding_rects:
        added_to_group = False
        for group in grouped_rects:
            if any(boxes_in_vicinity(rect, other, proximity_threshold) for other in group):
                group.append(rect)
                added_to_group = True
                break
        if not added_to_group:
            grouped_rects.append([rect])

    # Gộp các nhóm thành một hộp
    final_boxes = []
    for group in grouped_rects:
        if len(group) == 1:
            final_boxes.append(group[0])
        else:
            x_vals = [r[0] for r in group]
            y_vals = [r[1] for r in group]
            w_vals = [r[2] for r in group]
            h_vals = [r[3] for r in group]

            x_min = min(x_vals)
            y_min = min(y_vals)
            x_max = max([x + w for x, w in zip(x_vals, w_vals)])
            y_max = max([y + h for y, h in zip(y_vals, h_vals)])
            
            merged_w = x_max - x_min
            merged_h = y_max - y_min
            
            final_boxes.append((x_min, y_min, merged_w, merged_h))

    # Vẽ hộp chữ nhật
    img_copy = img.copy()
    for box in final_boxes:
        top_left_corner = (box[0], box[1])
        bottom_right_corner = (box[0] + box[2], box[1] + box[3])
        cv2.rectangle(img_copy, top_left_corner, bottom_right_corner, (0, 255, 0), 2)

    print(f"Số kết quả khớp (sau khi nhóm các hộp gần nhau): {len(final_boxes)}")
    img_rgb = cv2.cvtColor(img_copy, cv2.COLOR_BGR2RGB)

    # Hiển thị kết quả
    plt.figure(figsize=(10, 10))
    plt.imshow(img_rgb)
    plt.title("Kết quả Template Matching")
    plt.axis('off')
    plt.show()
match_template('Input.jpg', 'Temp.jpg')
